In [18]:
train = pd.read_csv('../data/train_v1.csv')
train['FinelineNumber'] = train['FinelineNumber'].astype(int)
train.head()

,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,company_code,product_code,Count
0,999,5,Friday,-1,FINANCIAL SERVICES,1000,81131,52929,0
1,30,7,Friday,1,SHOES,8931,05388,15980,1
2,30,7,Friday,1,PERSONAL CARE,4504,74108,11099,1
3,26,8,Friday,2,PAINT AND ACCESSORIES,3565,22384,03510,2
4,26,8,Friday,2,PAINT AND ACCESSORIES,1017,20066,13744,2


In [19]:
test = pd.read_csv('../data/test_v1.csv')
test = test.fillna(value = 9999)
test['FinelineNumber'] = test['FinelineNumber'].astype(int)
test.head()


,VisitNumber,Weekday,ScanCount,DepartmentDescription,FinelineNumber,company_code,product_code
0,1,Friday,1,SHOES,3002,25033,89714
1,1,Friday,1,DAIRY,1526,17077,10732
2,1,Friday,1,DAIRY,1431,94700,01026
3,1,Friday,1,GROCERY DRY GOODS,3555,84912,11470
4,2,Friday,1,DSD GROCERY,4408,28400,15224


In [22]:
df = train[['TripType','VisitNumber', 'FinelineNumber']]
df_ = test[['VisitNumber', 'FinelineNumber']]

In [38]:
type_list = list(df['TripType'].unique())
type_list.sort()

In [27]:
tmp_df = df[df['TripType'].isin([3])]
tmp_fine_list = list(tmp_df['FinelineNumber'].unique())

In [80]:
def get_fineline_set(df_f, type_list):

    set_list = []
    
    for i in type_list:
        td = df[df['TripType'].isin([i])]
        td_fine = list(td['FinelineNumber'].unique())
        set_list.append(td_fine)
    
    return set_list

In [114]:
def get_type_fineline_unique(set_list,type_list):

    unique_list = []
    
    for i in range(len(type_list)):
        t_type = type_list[i]
        tmp_unique = set_list[i]
        for j in range(len(set_list)):

            if type_list[j] != t_type:
                tmp_unique = list(set(tmp_unique) - set(set_list[j]))
                
        unique_list.append(tmp_unique)
    return unique_list

In [115]:
unique_list = get_type_fineline_unique(set_list,type_list)

In [156]:
def get_unique_fine_columns(df, unique_list):
    
    tmp_df  = df[['VisitNumber','Weekday']].drop_duplicates()
    
    for i in unique_list:
        t_df = df[df['FinelineNumber'] == i]
        t_df = t_df.groupby(['VisitNumber'], as_index = False)['ScanCount'].sum()
        t_df.rename(columns={'ScanCount': 'fine_%s' %(i)}, inplace = True)
        
        tmp_df = tmp_df.merge(t_df, how = 'left', on = 'VisitNumber', copy = True)
        tmp_df['fine_%s' %(i)].fillna(value = 0, inplace = True)
    
    tmp_df.drop(columns=['Weekday'], inplace = True)
    
    return tmp_df

In [158]:
train_f = get_unique_fine_columns(train, unique_fine)
test_f = get_unique_fine_columns(test, unique_fine)

In [159]:
df = train[['TripType','VisitNumber', 'company_code']]
df_ = test[['VisitNumber', 'company_code']]

# company_code

In [160]:
def get_company_set(df, type_list):

    set_list = []
    
    for i in type_list:
        td = df[df['TripType'].isin([i])]
        td_fine = list(td['company_code'].unique())
        set_list.append(td_fine)
    
    return set_list

In [161]:
company_set_list = get_company_set(df, type_list)

In [162]:
def get_type_company_unique(set_list,type_list):

    unique_list = []
    
    for i in range(len(type_list)):
        t_type = type_list[i]
        tmp_unique = set_list[i]
        for j in range(len(set_list)):

            if type_list[j] != t_type:
                tmp_unique = list(set(tmp_unique) - set(set_list[j]))
                
        unique_list.append(tmp_unique)
    return unique_list

In [164]:
company_unique_list = get_type_fineline_unique(company_set_list, type_list)

In [166]:
def get_unique_com_columns(df, unique_list):
    
    tmp_df  = df[['VisitNumber','Weekday']].drop_duplicates()
    
    for i in unique_list:
        t_df = df[df['company_code'] == i]
        t_df = t_df.groupby(['VisitNumber'], as_index = False)['ScanCount'].sum()
        t_df.rename(columns={'ScanCount': 'comcode_%s' %(i)}, inplace = True)
        
        tmp_df = tmp_df.merge(t_df, how = 'left', on = 'VisitNumber', copy = True)
        tmp_df['comcode_%s' %(i)].fillna(value = 0, inplace = True)
    
    tmp_df.drop(columns=['Weekday'], inplace = True)
    
    return tmp_df

In [170]:
c_unique_list = []

for i in company_unique_list:
    for j in i:
        c_unique_list.append(j)

In [172]:
len(c_unique_list)

810

In [174]:
train_c = get_unique_com_columns(train, c_unique_list)
test_c = get_unique_com_columns(test, c_unique_list)

In [175]:
df = train[['TripType','VisitNumber', 'product_code']]
df_ = test[['VisitNumber', 'product_code']]

In [176]:
def get_product_set(df, type_list):

    set_list = []
    
    for i in type_list:
        td = df[df['TripType'].isin([i])]
        td_fine = list(td['product_code'].unique())
        set_list.append(td_fine)
    
    return set_list

In [177]:
product_set_list = get_product_set(df, type_list)

In [178]:
def get_type_product_unique(set_list,type_list):

    unique_list = []
    
    for i in range(len(type_list)):
        t_type = type_list[i]
        tmp_unique = set_list[i]
        for j in range(len(set_list)):

            if type_list[j] != t_type:
                tmp_unique = list(set(tmp_unique) - set(set_list[j]))
                
        unique_list.append(tmp_unique)
    return unique_list

In [179]:
product_unique_list = get_type_product_unique(product_set_list, type_list)

In [180]:
p_unique_list = []

for i in product_unique_list:
    for j in i:
        p_unique_list.append(j)

In [181]:
len(p_unique_list)

15689

In [ ]:
def get_unique_product_columns(df, unique_list):
    
    tmp_df  = df[['VisitNumber','Weekday']].drop_duplicates()
    
    for i in unique_list:
        t_df = df[df['product_code'] == i]
        t_df = t_df.groupby(['VisitNumber'], as_index = False)['ScanCount'].sum()
        t_df.rename(columns={'ScanCount': 'prod_%s' %(i)}, inplace = True)
        
        tmp_df = tmp_df.merge(t_df, how = 'left', on = 'VisitNumber', copy = True)
        tmp_df['prod_%s' %(i)].fillna(value = 0, inplace = True)
    
    tmp_df.drop(columns=['Weekday'], inplace = True)
    
    return tmp_df

In [ ]:
train_p = get_unique_fine_columns(train, p_unique_list)
test_p = get_unique_fine_columns(test, p_unique_list)

In [182]:
print(train_f.shape, train_c.shape)
print(test_f.shape, test_c.shape)

(95674, 341) (95674, 811)
(95674, 341) (95674, 811)


In [186]:
train_f.to_csv('../data/unique_fine.csv', index = False)
test_f.to_csv('../data/t_unique_fine.csv', index = False)

train_c.to_csv('../data/unique_com.csv', index = False)
test_c.to_csv('../data/t_unique_com.csv', index = False)